In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, norm
from sklearn.utils import resample
from sklearn.metrics import ndcg_score

In [ ]:
bench = pd.read_csv('./ap_benchmarking_template.csv.gz')
bench_len = bench.shape[0]

In [ ]:
import os
os.chdir('./ap_models3/selected_models/')

## create ensembles and add to benchmark

In [ ]:
fold0_1 = pd.read_csv('')
fold0_2 = pd.Series.from_csv('')

fold1_1 = 
fold1_2 = 

fold2_1 = 
fold2_2 = 

fold3_1 = 
fold3_2 = 

top1_1 = 
top1_2 = 
top1_3 = 
top1_4 = 

top2_1 = 
top2_2 = 
top2_3 = 
top2_4 = 

In [ ]:
#  Define Ensembles

complete_ensemble = []
ensemble_folds = []
ensemble_tops = []
ensemble_fold_best = []
ensemble_top_best = []
ensemble_foldtop_best = []

In [ ]:
#  Add Ensembles to models 

bench['complete_ensemble'] = complete_ensemble
bench['ensemble_folds'] = ensemble_folds
bench['ensemble_tops'] = ensemble_tops
bench['ensemble_fold_best'] = ensemble_fold_best
bench['ensemble_top_best'] = ensemble_top_best
bench['ensemble_foldtop_best'] = ensemble_foldtop_best

models = ["netmhcpan4.el", "mhcflurry2.ap.with_flanks", 'complete_ensemble', 'ensemble_folds', 
          'ensemble_tops', 'ensemble_fold_best', 'ensemble_top_best', 'ensemble_foldtop_best']

## define stat functions

In [ ]:
def getPPV(data, k):
    name = data.columns[1]
    sort = data.sort_values(by=name)
    avg = sort.iloc[:k, 'hit'].mean()
    return ppv
    
    
def getNDCG(data, k):
    name = data.columns[1]
    true = data.loc[:, 'hit']
    pred = data.loc[:, name]
    ndcg = ndcg_score(true, pred, k=k)
    return ndcg


def bootstrap(bench, ppv_values, ndcg_values, sample_size, iterations, models, k_vals):
    for i in range(iterations):
        if i % 500 == 0:
            print("Iteration: " + str(i))
            
        sample = resample(df, n_samples=sample_size)
        for k in k_vals:
            for model in models:
                data = sample.loc[:, ['hit', model]]
                ppv_values[model + '_' + str(k)].append(get_PPV(data, k))
                ndcg_values[model + '_' + str(k)].append(get_NDCG(data, k))
                
    return ppv_values, ndcg_values

def fill_df(values_dict, df, comp1, comp2):
    for key in values_dict.keys()
        meta = key.split('_')
        k = int(meta[-1])
        mod_name = meta[:-1]
        mod_avg = mod_name + '_mean'
        mod_ci = mod_name + '_95CI'
        mod_p1 = mod_name + '_pvalue_pan'
        mod_p2 = mod_name + '_pvalue_flurry'
        
        data = np.asarray(values_dict[key])
        compare1 = np.asarray(values_dict[comp1 + '_' + str(k)])
        compare2 = np.asarray(values_dict[comp2 + '_' + str(k)])
        
        avg, sigma = np.mean(data), np.std(data)
        ci = norm.interval(0.95, loc=avg, scale=sigma/sqrt(N))[0]
        
        p1 = ttest_ind(data, compare1).pvalue
        p2 = ttest_ind(data, compare1).pvalue
        
        df.loc[k, mod_avg] = avg
        df.loc[k, mod_ci] = avg - ci
        df.loc[k, mod_p1] = p1
        df.loc[k, mod_p2] = p2
        
        return df

In [2]:
models = ["netmhcpan4.el", "mhcflurry2.ap.with_flanks", 'complete_ensemble', 'ensemble_folds', 
          'ensemble_tops', 'ensemble_fold_best', 'ensemble_top_best', 'ensemble_foldtop_best']

## make empty dfs and define k values

In [3]:
k = [10, 25, 50, 100, 250, 500]
n_samples = 100000
iters = 5000

In [4]:
zeros_total = np.zeros((len(k), 8))
ppv = pd.DataFrame(zeros_total, columns=models)
ndcg = pd.DataFrame(zeros_total, columns=models)

In [10]:
zeros_boot = np.zeros((len(k), 32))

avg_mods = [model + '_mean' for model in models]
ci_mods = [model + '_95CI' for model in models]
pval1_mods = [model + '_pvalue_pan' for model in models]
pval2_mods = [model + '_pvalue_flurry' for model in models]

boot_models = []
for i in range(len(avg_mods)):
    boot_models.append(avg_mods[i])
    boot_models.append(ci_mods[i])
    boot_models.append(pval1_mods[i])
    boot_models.append(pval2_mods[i])

boot_ppv = pd.DataFrame(zeros_boot, columns=boot_models)
boot_ndcg = pd.DataFrame(zeros_boot, columns=boot_models)

In [ ]:
#  make output dict for bootstrapping

ppv_values = dict()
ndcg_values = dict()
for i in k:
    for model in models:
        ppv_values[model + '_' + str(i)] = list()
        ndcg_values[model + '_' + str(i)] = list()

## Precision and NDCG at k

In [ ]:
#  Calculate for entire dataset

for i in k:
    for model in models:
        data = bench.loc[:, ['hit', model]]
        ppv.loc[i, model] = getPPV(data, k)
        ndcg.loc[i, model] = getNDCG(data, k)

In [ ]:
#  Bootstrap Resampling

ppv_values, ndcg_values = bootstrap(bench, ppv_values, ndcg_values, n_samples, iters, models, k_vals)

In [ ]:
#  Calc Mean, 95% CI, and p_value (comapred to mhcflurry2.0 AP)

boot_ppv = fill_df(ppv_values, boot_ppv, "netmhcpan4.el", "mhcflurry2.ap.with_flanks")
boot_ndcg = fill_df(ndcg_values, boot_ndcg,"netmhcpan4.el", "mhcflurry2.ap.with_flanks")

## write to excel file 

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('mhc_rank_benchmarking_rnd3.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
boot_ppv.to_excel(writer, sheet_name='bootstrapped_precision_at_k')
ppv.to_excel(writer, sheet_name='total_precision_at_k')

boot_ndcg.to_excel(writer, sheet_name='bootstrapped_ndcg_at_k')
ndcg.to_excel(writer, sheet_name='total_ndcg_at_k')

# Close the Pandas Excel writer and output the Excel file.
writer.save()